# Семинар 11

## Работа с текстовыми файлами (продолжение)

### Запись файлов

Если для чтения файлов используется метод `.read()`, то для **записи файлов** — метод `.write()`. Его нужно вызывать от объекта-файла и подавать в него строку, которую мы хотим записать в файл. Делать это нужно в режиме `"w"`:

In [1]:
text = "Ура! Наконец-то я, текст, буду в файле!"

with open("new_file.txt", "w", encoding="utf-8") as f:
    f.write(text)

Проверим, что текст сохранился:

In [2]:
with open("new_file.txt", "r", encoding="utf-8") as f:
    print(f.read())

Ура! Наконец-то я, текст, буду в файле!


**Внимание**! В режиме записи (`"w"`) файл всегда перезаписывается с нуля. Когда вы вызываете функцию `open()` в режиме записи (`"w"`), файл перезаписывается как пустой. Заметьте, что это происходит не в тот момент, когда вы вызываете `f.write()`! То есть даже если после подключения к файлу ничего не происходит, он всё равно перезаписывается:

In [3]:
with open("new_file.txt", "w", encoding="utf-8") as f:
    nenuzhnaya_peremennaya = True

with open("new_file.txt", "r", encoding="utf-8") as f:
    content = f.read()
print(content)
print(type(content))
print(len(content))


<class 'str'>
0


Если попытаться записать файл в режиме чтения `"r"`, то ничего не получится:

In [4]:
with open("new_file.txt", "r", encoding="utf-8") as f:
    f.write(text)

UnsupportedOperation: not writable

То же самое — если попытаться прочитать файл в режиме записи `"w"`:

In [5]:
with open("new_file.txt", "w", encoding="utf-8") as f:
    print(f.read())

UnsupportedOperation: not readable

## Обработка естественного языка

Сфера практических задач, которые связаны с обработкой текстовых материалов, обычно именуется **«(автоматическая) обработка естественного языка»**, или сокращённо «автобрея», а также ***Natural Language Processing***, или сокращённо NLP.

Вот самые распространённые и важные задачи NLP. Они нужны как в больших известных коммерческих программах, которые мы используем каждый день, так и в сугубо научных целях — для обработки текстов и других языковых данных:

- **токенизация**, то есть деление текста на токены
    - `"Маленькой ёлочке холодно зимой"` → `["маленькой", "ёлочке", "холодно", "зимой"]`
    - мы уже умеем делать токенизацию некоторыми простыми способами (например, сначала `text.split()`, а потом для каждого слова `word.strip()` — но научимся делать это более эффективно
- **лемматизация**, то есть приведение разных словоформ одной лексемы к лемме (начальной форме слова)
    - `"Маленькой ёлочке холодно зимой"` → `["маленький", "ёлочка", "холодно", "зима"]`
- **стемминг** — похоже на лемматизацию, но вместо леммы получается основа слова (англ. *stem*)
    - `"Маленькой ёлочке холодно зимой"` → `["маленьк", "ёлочк", "холодн", "зим"]`
- **частеречный тэггинг** (*POS-tagging*) — определение частей речи
    - `"Маленькой ёлочке холодно зимой"` → `["ADJ", "N", "ADV", "N"]`
- **выделение морфологических признаков** (*POS-tagging*) — определение падежа, числа, лица, времени и прочих грамматических признаков
    - `"Маленькой ёлочке холодно зимой"` → `[["SG", "FEM", "DAT"], ["SG", "DAT"], [], ["SG", "INS"]]`
- **синтаксический анализ** — выделение связей-зависимостей между словами / синтаксических групп

А также некоторые более продвинутые задачи (большинство из них для нас слишком сложны, но некоторые мы в этом курсе рассмотрим):
- **выделение частотных слов**
- **векторизация** — превращение текста из последовательности символов в *вектор* (последовательность чисел, которая хранит некоторую информацию о тексте)
- **классификация текстов** — например, определение жанра текста (сказка / текст закона / деловое письмо…), спама (спам / нормальное письмо), тональности текста (положительно окрашенный / нейтральный / отрицательный…), авторства (Маккартни / Леннон / Старр…)
- **спелл-чекинг**
- **извлечение информации** — извлечение из текста фактов, имён, названий, дат
- **суммаризация**, автоматический пересказ текста
- **машинный перевод** (в онлайн-переводчиках)
- **генерация текста** (в чат-ботах)

Подробнее про NLP можно прочитать в куче интернет-ресурсов, например, **[здесь](https://neerc.ifmo.ru/wiki/index.php?title=%D0%9E%D0%B1%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D0%B0_%D0%B5%D1%81%D1%82%D0%B5%D1%81%D1%82%D0%B2%D0%B5%D0%BD%D0%BD%D0%BE%D0%B3%D0%BE_%D1%8F%D0%B7%D1%8B%D0%BA%D0%B0)**.

### Регулярные выражения

Один из инструментов, который постепенно подводит нас к обработке естественного языка — это **регулярные выражения**, или попросту *регулярки* (англ. *regular expressions*, сокращённо *RE*). Регулярные выражения — это мини-язык программирования, который позволяет искать в тексте не только подстроки, но и некоторые «шаблоны»-маски. Например, с помощью регулярных выражения можно попросить: найди все сочетания согласного и гласного. Или все слова, которые начинаются на «био». Обычными способами мы можем найти только те подстроки, которые нам полностью известны, поэтому регулярки часто бывают полезны. Вот подробный **[гайд по регуляркам](https://habr.com/ru/articles/349860/#Primery_regulyarnyh_vyrazheniy)**.

Чтобы использовать регулярки в питоне, нужно импортировать модуль **`re`**:

In [6]:
import re

Загрузим известный нам текст про Гарри Поттера:

In [7]:
with open("sem10-hpps_ru_ch1.txt", "r", encoding="utf-8") as f:
    text = f.read()

print(text[:500])

Мистер и миссис Дурсль, живший в доме номер четыре по Бирючинной улице, с гордостью говорили, что они, слава богу, совершенно нормальны. Они были последними людьми, от которых можно было ожидать чего-то странного или таинственного, потому что они просто не держались за такую ерунду.

Мистер Дурсль был директором фирмы «Граннингс», которая производила дрели. Это был крупный, мускулистый мужчина, почти без шеи имевший очень большие усы. Миссис Дурсль была худой блондинкой с почти вдвое более длино


_____

#### Функции `re.search()` и `re.finditer()`

Функция **`re.search()`** из модуля `re` поможет найти первое вхождение какой-то подстроки в текст. В неё подаётся два аргумента: подстрока, которую нужно найти, и строка, в которой её нужно найти:

In [8]:
print(re.search("Поттер", text))

<re.Match object; span=(919, 925), match='Поттер'>


Если `re.search()` не может ничего найти, она возвращает объект `None`:

In [9]:
print(re.search("Боромир", text))

None


Другая функция, **`re.finditer()`**, находит не только первое вхождение, а все вхождения подстроки — в виде специального итерируемого объекта (который можно превратить в список функцией `list()`):

In [10]:
list(re.finditer("Поттер", text))

[<re.Match object; span=(919, 925), match='Поттер'>,
 <re.Match object; span=(936, 942), match='Поттер'>,
 <re.Match object; span=(1238, 1244), match='Поттер'>,
 <re.Match object; span=(1285, 1291), match='Поттер'>,
 <re.Match object; span=(1421, 1427), match='Поттер'>,
 <re.Match object; span=(5041, 5047), match='Поттер'>,
 <re.Match object; span=(5484, 5490), match='Поттер'>,
 <re.Match object; span=(5571, 5577), match='Поттер'>,
 <re.Match object; span=(8612, 8618), match='Поттер'>,
 <re.Match object; span=(9404, 9410), match='Поттер'>,
 <re.Match object; span=(10125, 10131), match='Поттер'>,
 <re.Match object; span=(10441, 10447), match='Поттер'>,
 <re.Match object; span=(10534, 10540), match='Поттер'>,
 <re.Match object; span=(17055, 17061), match='Поттер'>,
 <re.Match object; span=(17096, 17102), match='Поттер'>,
 <re.Match object; span=(17482, 17488), match='Поттер'>,
 <re.Match object; span=(17633, 17639), match='Поттер'>,
 <re.Match object; span=(19181, 19187), match='Поттер'>

В результате работы этих функций мы получаем объекты типа `match`. Этот объект хранит информацию о том, какую строку мы нашли, и где в строке (на каких индексах строки подстрока начинается и заканчивается). Например, из результата поиска строки `"Поттер"` мы видим, что первое её вхождение находится в диапазоне от 919 до 925 символа строки `text`. Проверим, так ли это, с помощью среза:

In [11]:
print(text[919:925])
print(text[900:940])

Поттер
то-нибудь узнает о Поттерах. Миссис Потт


Впрочем, искать обычную подстроку мы и так умели (с помощью метода `str.find()`). В чём настоящая сила регулярок — это в **метасимволах**, которые позволяют ставить условия на поиск символов. Вот некоторые из них:

- `.` — любой символ
- `X?` — символ X, повторённый 0 или 1 раз
- `X*` — символ X, повторённый 0 раз или больше
- `X+` — символ X, повторённый 1 раз или больше
- `[ABC]` — любой из символов A, B и C
- `[^ABC]` — любой из символов, *кроме* A, B и C
- `[A-Z]` — любой из символов, расположенных между символами A и Z в [таблице Юникода](https://symbl.cc/ru/unicode-table/)
- `(AB|XY)` — либо последовательность символов AB, либо последовательность символов XY

Эти метасимволы можно комбинировать между собой. Таким образом мы можем попробовать найти в тексте, например, все прилагательные в единственном числе, мужском роде и именительном падеже. Для этого напишем регулярку, в которой будет:
1. корень прилагательного — последовательность из любых символов кириллического русского алфавита, повторённых 1 или больше раз: **`[а-яё]+`**
   - обратите внимание, что буква «ё» в Юникоде находится отдельно от всех остальных букв русской кириллицы, поэтому её нужно отдельно добавлять после диапазона `а-я`
2. окончание прилагательного: «ий», «ый» или «ой»: **`[иоы]й`**
   - альтернативный вариант: `(ий|ый|ой)` (делает то же самое)
  
Найдём все подстроки, в которых сначала идёт основа, а затем окончание:

In [12]:
list(re.finditer("[а-яё]+[иоы]й", "великий и могучий персидский язык — самый прекрасный в мире"))

[<re.Match object; span=(0, 7), match='великий'>,
 <re.Match object; span=(10, 17), match='могучий'>,
 <re.Match object; span=(18, 28), match='персидский'>,
 <re.Match object; span=(36, 41), match='самый'>,
 <re.Match object; span=(42, 52), match='прекрасный'>]

Как, используя регулярки, найти в тексте все словоформы русского слова? Есть несколько вариантов. Например, найдём сначала основу слова, а затем последовательность любых символов, кроме пробела:

In [13]:
list(re.finditer("Поттер[^ ]*", text))

[<re.Match object; span=(919, 928), match='Поттерах.'>,
 <re.Match object; span=(936, 942), match='Поттер'>,
 <re.Match object; span=(1238, 1245), match='Поттеры'>,
 <re.Match object; span=(1285, 1293), match='Поттеров'>,
 <re.Match object; span=(1421, 1429), match='Поттеров'>,
 <re.Match object; span=(5041, 5049), match='Поттеры,'>,
 <re.Match object; span=(5484, 5490), match='Поттер'>,
 <re.Match object; span=(5571, 5578), match='Поттер,'>,
 <re.Match object; span=(8612, 8631), match='Поттерах...\n\nМиссис'>,
 <re.Match object; span=(9404, 9412), match='Поттер".'>,
 <re.Match object; span=(10125, 10134), match='Поттерам?'>,
 <re.Match object; span=(10441, 10448), match='Поттеры'>,
 <re.Match object; span=(10534, 10541), match='Поттеры'>,
 <re.Match object; span=(17055, 17064), match='Поттеров.'>,
 <re.Match object; span=(17096, 17103), match='Поттеры'>,
 <re.Match object; span=(17482, 17491), match='Поттеров,'>,
 <re.Match object; span=(17633, 17641), match='Поттера,'>,
 <re.Match ob

Небольшая проблема в том, что к нашим словам «приклеились» знаки препинания — они-то ведь тоже не пробелы (а ещё слово «Миссис»). Для некоторых задач это хорошо, для других нам нужны только сами словоформы без пунктуации. Если нужны только словоформы, можно сделать иначе — описать все символы, которые могут быть в окончании. По идее, это все русские кириллические буквы (ну или можно перечислить все буквы, которые реально встречаются в окончаниях существительных):

In [14]:
list(re.finditer("Поттер[а-яё]*", text))

[<re.Match object; span=(919, 927), match='Поттерах'>,
 <re.Match object; span=(936, 942), match='Поттер'>,
 <re.Match object; span=(1238, 1245), match='Поттеры'>,
 <re.Match object; span=(1285, 1293), match='Поттеров'>,
 <re.Match object; span=(1421, 1429), match='Поттеров'>,
 <re.Match object; span=(5041, 5048), match='Поттеры'>,
 <re.Match object; span=(5484, 5490), match='Поттер'>,
 <re.Match object; span=(5571, 5577), match='Поттер'>,
 <re.Match object; span=(8612, 8620), match='Поттерах'>,
 <re.Match object; span=(9404, 9410), match='Поттер'>,
 <re.Match object; span=(10125, 10133), match='Поттерам'>,
 <re.Match object; span=(10441, 10448), match='Поттеры'>,
 <re.Match object; span=(10534, 10541), match='Поттеры'>,
 <re.Match object; span=(17055, 17063), match='Поттеров'>,
 <re.Match object; span=(17096, 17103), match='Поттеры'>,
 <re.Match object; span=(17482, 17490), match='Поттеров'>,
 <re.Match object; span=(17633, 17640), match='Поттера'>,
 <re.Match object; span=(19181, 191

Заметьте, что в примере выше мы используем звёздочку-астериск: `[а-яё]*`, а не плюсик `+`. Это потому, что звёздочка ищет последовательности от 0 и больше раз, а значит, она найдёт в том числе те подстроки, где окончание нулевое (то есть в том числе нулевую последовательность символов). А плюсик ищет от 1 и больше раз, так что с ним найдутся только *не*нулевые окончания. Тогда в выдаче не будет просто формы «Поттер»:

In [15]:
list(re.finditer("Поттер[а-яё]+", text))

[<re.Match object; span=(919, 927), match='Поттерах'>,
 <re.Match object; span=(1238, 1245), match='Поттеры'>,
 <re.Match object; span=(1285, 1293), match='Поттеров'>,
 <re.Match object; span=(1421, 1429), match='Поттеров'>,
 <re.Match object; span=(5041, 5048), match='Поттеры'>,
 <re.Match object; span=(8612, 8620), match='Поттерах'>,
 <re.Match object; span=(10125, 10133), match='Поттерам'>,
 <re.Match object; span=(10441, 10448), match='Поттеры'>,
 <re.Match object; span=(10534, 10541), match='Поттеры'>,
 <re.Match object; span=(17055, 17063), match='Поттеров'>,
 <re.Match object; span=(17096, 17103), match='Поттеры'>,
 <re.Match object; span=(17482, 17490), match='Поттеров'>,
 <re.Match object; span=(17633, 17640), match='Поттера'>,
 <re.Match object; span=(19633, 19640), match='Поттера'>,
 <re.Match object; span=(25278, 25285), match='Поттера'>]

Чтобы найти все словоформы фамилии «Дурсль», придётся сократить основу до «Дурсл-» (иначе не найдутся косвенные формы «Дурслей», «Дурслях» и прочие):

In [16]:
list(re.finditer("Дурсл[а-яё]*", text))

[<re.Match object; span=(16, 22), match='Дурсль'>,
 <re.Match object; span=(292, 298), match='Дурсль'>,
 <re.Match object; span=(446, 452), match='Дурсль'>,
 <re.Match object; span=(648, 655), match='Дурслей'>,
 <re.Match object; span=(749, 756), match='Дурслей'>,
 <re.Match object; span=(963, 969), match='Дурсль'>,
 <re.Match object; span=(1033, 1039), match='Дурсль'>,
 <re.Match object; span=(1178, 1184), match='Дурсли'>,
 <re.Match object; span=(1265, 1271), match='Дурсли'>,
 <re.Match object; span=(1517, 1523), match='Дурсль'>,
 <re.Match object; span=(1724, 1730), match='Дурсль'>,
 <re.Match object; span=(1803, 1809), match='Дурсль'>,
 <re.Match object; span=(1979, 1985), match='Дурсль'>,
 <re.Match object; span=(2021, 2027), match='Дурсль'>,
 <re.Match object; span=(2178, 2184), match='Дурсль'>,
 <re.Match object; span=(2381, 2387), match='Дурсль'>,
 <re.Match object; span=(2616, 2622), match='Дурсль'>,
 <re.Match object; span=(2682, 2688), match='Дурсль'>,
 <re.Match object; spa

#### Практические задачи

##### Задача 11.1

Как найти в `text` все слова, начинающиеся с большой буквы?

In [ ]:
# ваше решение

##### Задача 11.2

Как найти все слова с корнем «верн» («повернулась», «верну», «развернувшиеся»)?

In [ ]:
# ваше решение

##### Задача 11.3

Как найти все слова, обёрнутые кавычками `«»` (например, `«Фольксваген»`)?

In [ ]:
# ваше решение

##### Задача 11.4

Как найти все слова с дефисом (например, «как-нибудь», «Сами-Знаете-Кто»)?

In [ ]:
# ваше решение

#### Экранирование

В регулярках используются метасимволы — символы, имеющие специальное функциональное значение. Иногда это приводит к проблемам. Например, если мы хотим найти все слова, заканчивающиеся на точку, питон воспримет точку как метасимвол со значением «любой символ»:

In [17]:
spisok = list(re.finditer("[А-ЯЁа-яё]+.", text))
spisok[:10]

[<re.Match object; span=(0, 7), match='Мистер '>,
 <re.Match object; span=(7, 9), match='и '>,
 <re.Match object; span=(9, 16), match='миссис '>,
 <re.Match object; span=(16, 23), match='Дурсль,'>,
 <re.Match object; span=(24, 31), match='живший '>,
 <re.Match object; span=(31, 33), match='в '>,
 <re.Match object; span=(33, 38), match='доме '>,
 <re.Match object; span=(38, 44), match='номер '>,
 <re.Match object; span=(44, 51), match='четыре '>,
 <re.Match object; span=(51, 54), match='по '>]

Решением этой проблемы служит специальный механизм, называемый **экранированием**. Метасимволы можно *экранировать*, и тогда они будут восприниматься как обычные символы. Чтобы экранировать символ, нужно поставить перед ним обратную косую черту (бэкслэш) `\`. Тогда мы найдём только слова с точкой на конце:

In [18]:
spisok = list(re.finditer("[А-ЯЁа-яё]+\.", text))
spisok[:10]

[<re.Match object; span=(126, 136), match='нормальны.'>,
 <re.Match object; span=(276, 283), match='ерунду.'>,
 <re.Match object; span=(353, 359), match='дрели.'>,
 <re.Match object; span=(434, 438), match='усы.'>,
 <re.Match object; span=(636, 645), match='соседями.'>,
 <re.Match object; span=(735, 745), match='рождалось.'>,
 <re.Match object; span=(856, 865), match='раскроет.'>,
 <re.Match object; span=(919, 928), match='Поттерах.'>,
 <re.Match object; span=(1168, 1177), match='возможно.'>,
 <re.Match object; span=(1258, 1264), match='улице.'>]

Если использовать некоторые метасимволы без экранирования, может даже возникнуть ошибка. Например, если без экранирования поискать вопросительные знаки. Ошибка возникает из-за того, что метасимвол `?` используется для поиска 0 или 1 вхождения символа, который находится слева от него (например, `[аеиоу]?`) — а в нашем запросе слева от него ничего нету:

In [19]:
list(re.finditer("?", text))

error: nothing to repeat at position 0

Исправим с помощью экранирования:

In [20]:
spisok = list(re.finditer("\?", text))
spisok[:10]

[<re.Match object; span=(2573, 2574), match='?'>,
 <re.Match object; span=(7453, 7454), match='?'>,
 <re.Match object; span=(8055, 8056), match='?'>,
 <re.Match object; span=(8539, 8540), match='?'>,
 <re.Match object; span=(8557, 8558), match='?'>,
 <re.Match object; span=(8593, 8594), match='?'>,
 <re.Match object; span=(8836, 8837), match='?'>,
 <re.Match object; span=(9011, 9012), match='?'>,
 <re.Match object; span=(9159, 9160), match='?'>,
 <re.Match object; span=(9551, 9552), match='?'>]

#### Атрибуты объекта *match*

Научимся работать с объектом типа *match*. Как вынуть из него найденную подстроку? Или те самые индексы начала и конца подстроки в тексте? Если попробовать просто конвертировать этот объект в тип `str`, получится не очень:

In [21]:
result = re.search("Поттер[а-яё]+", text)

print(result)
print(type(result))

print(str(result))

<re.Match object; span=(919, 927), match='Поттерах'>
<class 're.Match'>
<re.Match object; span=(919, 927), match='Поттерах'>


На самом деле для этого есть особые методы:
- `.group()` — найденная подстрока (мы пока что не понимаем, почему она называется «group», но наверное поймём чуть позже)
- `.span()` — кортеж с двумя индексами: началом и концом подстроки
- `.start()` — индекс-начало подстроки
- `.end()` — индекс-конец подстроки

In [22]:
result_stroka = result.group()
print(result_stroka)
print(type(result_stroka))

Поттерах
<class 'str'>


In [23]:
result_span = result.span()
print(result_span)
print(type(result_span))

(919, 927)
<class 'tuple'>


In [24]:
print(result.start())
print(result.end())

919
927


#### Функции `re.findall()`, `re.sub()` и `re.split()`

Есть ещё несколько функций, которые помогут нам более эффективно работать с регулярками. К примеру, функция **`re.findall()`** похожа на `re.finditer()`, но возвращает сразу список со строками. Зачастую это легче, чем получать из `re.finditer()` итератор и из каждого элемента извлекать `.group()`. Но так, к сожалению, мы не видим индексы вхождений:

In [25]:
re.findall("Поттер[а-яё]+", text)

['Поттерах',
 'Поттеры',
 'Поттеров',
 'Поттеров',
 'Поттеры',
 'Поттерах',
 'Поттерам',
 'Поттеры',
 'Поттеры',
 'Поттеров',
 'Поттеры',
 'Поттеров',
 'Поттера',
 'Поттера',
 'Поттера']

А ещё есть уже знакомые нам функции в новом обличье. Например, мы знаем метод `str.replace()`, который помогал заменять одни подстроки на другие, а теперь ещё познакомимся с функцией **`re.sub()`** (от англ. *substitute* ‘заменить’). Она делает то же самое, что `str.replace()`, но в ней можно использовать регулярки. С помощью неё можно, например, вычеркнуть Дурслей из истории и заменить все вхождения этой фамилии на нижнее подчёркивание. В функцию сначала подаётся строка, которую надо найти, потом строка-замена, и в конце — строка, в которой надо искать (для краткости я вывожу на экран только первые 1000 символов):

In [26]:
print(re.sub("Дурсл[а-яё]+", "____", text)[:1000])

Мистер и миссис ____, живший в доме номер четыре по Бирючинной улице, с гордостью говорили, что они, слава богу, совершенно нормальны. Они были последними людьми, от которых можно было ожидать чего-то странного или таинственного, потому что они просто не держались за такую ерунду.

Мистер ____ был директором фирмы «Граннингс», которая производила дрели. Это был крупный, мускулистый мужчина, почти без шеи имевший очень большие усы. Миссис ____ была худой блондинкой с почти вдвое более длиной шеей, чем обычно, и это было очень кстати, поскольку она проводила много времени, перегибаясь через садовые изгороди и подглядывая за соседями. У ____ был маленький сын по имени Дадли, и, по их мнению, лучшего мальчика никогда не рождалось.

У ____ было все, что они хотели, но у них также была тайна, и больше всего они боялись, что кто-нибудь ее раскроет. Они думали, что не вынесут, если кто-нибудь узнает о Поттерах. Миссис Поттер была сестрой миссис ____, но они не встречались уже несколько лет; на

Ещё один улучшенный вариант известного нам инструмента — функция **`re.split()`**. Это то же самое, что `str.split()`, но в ней можно использовать регулярки. К примеру, разделим текст по знакам препинания: запятым, точкам и так далее:

In [27]:
re.split("[,\.\!\?\n]+", text)[:30]

['Мистер и миссис Дурсль',
 ' живший в доме номер четыре по Бирючинной улице',
 ' с гордостью говорили',
 ' что они',
 ' слава богу',
 ' совершенно нормальны',
 ' Они были последними людьми',
 ' от которых можно было ожидать чего-то странного или таинственного',
 ' потому что они просто не держались за такую ерунду',
 'Мистер Дурсль был директором фирмы «Граннингс»',
 ' которая производила дрели',
 ' Это был крупный',
 ' мускулистый мужчина',
 ' почти без шеи имевший очень большие усы',
 ' Миссис Дурсль была худой блондинкой с почти вдвое более длиной шеей',
 ' чем обычно',
 ' и это было очень кстати',
 ' поскольку она проводила много времени',
 ' перегибаясь через садовые изгороди и подглядывая за соседями',
 ' У Дурслей был маленький сын по имени Дадли',
 ' и',
 ' по их мнению',
 ' лучшего мальчика никогда не рождалось',
 'У Дурслей было все',
 ' что они хотели',
 ' но у них также была тайна',
 ' и больше всего они боялись',
 ' что кто-нибудь ее раскроет',
 ' Они думали',
 ' что не в

#### Практические задачи

##### Задача 11.5

Как найти в `text` все слова *и выражения*, обёрнутые кавычками `«»` (например, `«Фольксваген»`, `«Сами-Знаете-Кто»`, `«Всемирная сеть»`)?

In [ ]:
# ваше решение

##### Задача 11.6

Сколько в `text` форм глагола «быть» (включая формы прошедшего времени «была», «были», «было», «был», формы будущего времени «буду», «будем», «будешь» и т.д. и императив «будь»)? Подсчитайте количество и выведите все их на экран.

(Авторский ответ: **96**.)

*Заметьте, что в выдаче наверняка будет несколько ошибочных результатов — например, слово «будто». Мы не можем отсеять их регулярками (точнее, можем, но это сложно), но потом изучим другие методы для более качественного поиска.*

In [ ]:
# ваше решение

##### Задача 11.7

Сделаем учебный материал для изучающих русский язык, где им нужно будет самим поставить глагол в нужную форму в зависимости от контекста. Для этого замените в `text` все формы глагола «быть» на форму «БЫТЬ» капсом в скобках:
- `было` → `(БЫТЬ)`
- `буду` → `(БЫТЬ)`
- …

Выведите на экран первые 1000 символов получившегося изменённого текста.

In [ ]:
# ваше решение

##### Задача 11.8

В японском языке очень простая слоговая структура: **(C)V(n)**. Слог может состоять или только из гласного (V), или из согласного и гласного (CV), а ещё в конце любого слога может быть согласный /n/ (остальные согласные не могут быть в конце). К примеру, этим правилам следуют японские слова *a-ri-ga-to* ‘спасибо’, *Nin-ten-do* ‘Нинтендо’, *su-mi-ma-sen* ‘извините’, *fu-ji* ‘Фудзи’. Но им не следуют всякие не-японские слова, например, английские *hello*, *goat*, *spray*. (Это несколько упрощённая картина японской фонологии — в ней нет удвоенных согласных, дифтонгов и ещё кое-чего, но мы ограничимся ей для простоты.)

Напишите функцию `can_be_japanese()`, которая будет принимать на вход слово и определять, может ли оно (в теории) быть японским словом — судя по его слоговой структуре. Эта функция должна выдать `True` на все японские примеры выше и `False` на все не-японские примеры выше. В рамках задачи будем считать согласными и гласными символы стандартной латиницы (которых, конечно, недостаточно для записи настоящего японского). Они уже даны вам в виде строк ниже.

In [ ]:
consonants = "bcdfghjklmnprstqvwxyz"
vowels = "aeiou"

# ваше решение
    


################################

for stroka in ("arigato", "nintendo", "sumamisen", "fuji", "hello", "goat", "spray"):
    print(stroka)
    print(can_be_japanese(stroka))

##### Задача 11.9

Напишите функцию `find_dates()`, которая будет:
- принимать на вход текст (типа `str`)
- искать в тексте все подстроки типа `"DD-MM-YYYY"`, которые могут быть датами формата `день-месяц-год`
    - обратите внимание: функция должна находить любые годы нашей эры (от 1 года н. э. до очень больших чисел из будущего, типа 99999 год н. э.)
    - нужно учитывать, что не все последовательности чисел формата `"XX-XX-XXXX"` могут быть настоящими датами (см. пример с IP-адресом ниже)
    - однозначные дни и номера месяцев могут записываться с нулём в начале, а могут без (март может записываться как `03` или как `3`)
- возвращать список с этими датами (в виде строк)

In [ ]:
text = """Как говорят, 10.06.754 начал править Аль-Мансур, второй аббасидский халиф. 
5.3.1953 умер Сталин. 10.02.2025 проходит наша пара. 
А вот 64.64.64 — это часть моего IPv4-адреса. А 96.98 — это цена доллара."""
print(text)

Ожидаемый результат работы функции:
```
> find_dates(text)
["10.06.754", "5.3.1953", "10.02.2025"]
```

In [ ]:
# ваше решение



################################

print(find_dates(text))